In [1]:
from agential.cog.reflexion.strategies.qa import ReflexionReActQAStrategy
from agential.llm.llm import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

strat = ReflexionReActQAStrategy(llm=llm)

In [2]:
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_INSTRUCTION_HOTPOTQA, REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA


question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

out = strat.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=3,
    reset=True,
)

<PROMPT AGENT=============================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episo